### Multi-Tool Calling with MAF and Azure AI Agent Service

![multi-tool-agent](./images/multi-tool-agent.png)

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4 jsonref==1.1.0

### Setting up the Environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Loading the Code Interpreter Tool

In [ ]:
from agent_framework import AgentRunResponse, ChatResponseUpdate, HostedCodeInterpreterTool

code_interpreter_tool = HostedCodeInterpreterTool()


### Loading the Hosted MCP Tool

In [ ]:
from agent_framework import HostedMCPTool

ms_learn_mcp_tool = HostedMCPTool(
    name = "Microsoft Learn MCP Tool",
    url = "https://learn.microsoft.com/api/mcp"
)

### Creating the Agent with Multiple Tools in Azure AI Agent Service 

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

# Cell 2: Define async workflow
async def create_multi_tool_agent():
    # Create async Azure credential
    credential = AzureCliCredential()
    
    # Initialize the Azure AI Agent Client
    chat_client = AzureAIAgentClient(async_credential=credential,
                                     project_endpoint=project_endpoint,
                                     model_deployment_name=model)

    try:

        # Create the Multi-Tool Agent
        agent = chat_client.create_agent(
            name="multi-tool-agent",
            instructions="You are a helpful multi-tool agent.",
            tools=[code_interpreter_tool, ms_learn_mcp_tool]
        )

        print("✅ Agent created successfully!")
        return agent

    finally:
        # Clean up async clients
        await chat_client.close()
        await credential.close()


In [ ]:
# Cell 3: Run the async function
agent = await create_multi_tool_agent()


### Creating an Async Function to Handle User Approvals

In [ ]:
from agent_framework import ChatMessage

async def handle_approvals_with_thread(query: str, agent, thread):
    result = await agent.run(query, thread=thread, store=True)
    while len(result.user_input_requests) > 0:
        new_input = []
        for user_input_needed in result.user_input_requests:
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(True)],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    return result


### Running Different Queries on the Agent

In [ ]:
thread = agent.get_new_thread()
query = "Can you write a Python function to calculate the factorial of a number using recursion and give me the output for number 100"
response = await handle_approvals_with_thread(query, agent, thread)

print("\n🧠 Assistant:\n", response)


In [ ]:
thread = agent.get_new_thread()
query = "How to create an Azure storage account using az cli?"
response = await handle_approvals_with_thread(query, agent, thread)

print("\n🧠 Assistant:\n", response)
